Librerias

In [2]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import re
# Configuración warnings
# ==============================================================================\n",
import warnings
warnings.filterwarnings('ignore')

Funciones

In [3]:
def GetIssuesSiteGithub(url,repo,proy,tipo):
    pr = LeerurlIssues(url,repo,proy,tipo)
    df_Final = IssuesToDataFrame(pr)
    return df_Final
###
## Funcion que lee de un proyecto los todos los issues abiertos y cerrados.
## con fines, educaciones y te prueba de concepto.
###
def LeerurlIssues(url,repo,proy,tipo):
    #Rutas Fijas para identificar Issues Abiertos y Cerrados
    urlabiertos = '/issues?q=is%3Aopen+is%3Aissue'
    urlcerrados = '/issues?q=is%3Aissue+is%3Aclosed'
    patronbusqueda = repo+proy+'/'+tipo+'/'

    
    soupopen = leerUrlParametros(url,repo,proy,urlabiertos)
    linksopen = soupopen('a')
    
    dfabiertos = ExtraerLinkIssues(url,linksopen,patronbusqueda)
    soupclosed = leerUrlParametros(url,repo,proy,urlcerrados)
    linksclosed = soupclosed('a')
    dfcerrados = ExtraerLinkIssues(url,linksclosed,patronbusqueda)
    
    df = pd.concat([dfabiertos, dfcerrados], axis=0)
    return df  

def ExtraerLinkIssues(url,linkPage,patron):
    lista = []
    for tag in linkPage:
        valor = tag.get('href')
        if(valor.find(patron) != -1):
            
            lista.append(url+ valor)        
    df = pd.DataFrame (lista, columns = ['url_issues'])
    df = df.drop_duplicates()
    return df

def leerUrlParametros(url,repo,proy,tipo):      
    pagina =   url+repo+proy+tipo
    soup = BeautifulSoup(urllib.request.urlopen(pagina).read().decode())
    return  soup

def leerUrl(pagina):    
    soup = BeautifulSoup(urllib.request.urlopen(pagina).read().decode())
    return  soup

def IssuesToDataFrame(lista):
    columns = ["issue", "author", "state", "assignees"]
    df = pd.DataFrame(columns=columns)

    for i in range(len(lista)):
        link = str(lista.iloc[i]['url_issues'])        
        d = ParsearPagina(link)
        df = pd.concat([df, d], axis=0)
    return df


def ParsearPagina (link):
    parts = str(link).split('/')
    issues = parts[len(parts)-1]
    page = leerUrl(link)
    # Con el DataFrame de las paginas de Issues se procede a convertirlo en el DataFrame tarea
    # definición del DataFrame
    columns = ["issue", "author", "state", "assignees"]
    df = pd.DataFrame(columns=columns)
    state = "open"
    page.find_all(content= re.compile(link +"$"))
    if page.find("span", attrs={"title": "Status: Closed"}):
        state = "closed"
    state
    author = page.find("a", attrs={"class": "author text-bold Link--secondary"}).get_text()
    author
    assignees = (
            page.find("span", attrs={"class": "css-truncate js-issue-assignees"})
            .get_text()
            .replace("\n", "")
            .split()
        )

    if assignees == ["No", "one", "assigned"]:
        assignees = ["No one assigned"]
    assignees
    df = df.append(
                {"issue": issues, "author": author, "state": state, "assignees": assignees},
                ignore_index=True,
            )
    return df




In [4]:
PaginaRepositorio = '/pescap'
UrlBase = 'https://github.com'
Proyecto = '/WebScrapingMDS'
TipoGitHub = 'issues'
df = GetIssuesSiteGithub(UrlBase,PaginaRepositorio,Proyecto,TipoGitHub)


In [5]:
df.sort_values(by='issue' , ascending=True)

,issue,author,state,assignees
0,1,pescap,open,[No one assigned]
0,15,DenisOsses,closed,[No one assigned]
0,17,CarlosZuritaEstolaza,open,[CarlosZuritaEstolaza]
0,2,pescap,closed,"[pescap, caspoerer]"
0,20,CarlosZuritaEstolaza,open,[No one assigned]
0,21,gavalenz,open,[No one assigned]
0,24,kevimulloa,open,[No one assigned]
0,25,josemoragonzalez,closed,"[pescap, josemoragonzalez]"
0,27,pescap,open,[No one assigned]
0,28,JoseAlexanderCL,closed,[JoseAlexanderCL]


In [6]:
PaginaRepositorio = '/pescap'
UrlBase = 'https://github.com'
Proyecto = '/WebScrapingMDS'
TipoGitHub = 'issues'

#page = leerUrlParametros(UrlBase,PaginaRepositorio,Proyecto,TipoGitHub)

pr = LeerurlIssues(UrlBase,PaginaRepositorio,Proyecto,TipoGitHub)
print(pr)


                                           url_issues
0   https://github.com/pescap/WebScrapingMDS/issue...
2   https://github.com/pescap/WebScrapingMDS/issue...
4   https://github.com/pescap/WebScrapingMDS/issue...
6   https://github.com/pescap/WebScrapingMDS/issue...
9   https://github.com/pescap/WebScrapingMDS/issue...
11  https://github.com/pescap/WebScrapingMDS/issue...
14  https://github.com/pescap/WebScrapingMDS/issue...
17  https://github.com/pescap/WebScrapingMDS/issue...
20  https://github.com/pescap/WebScrapingMDS/issue...
23  https://github.com/pescap/WebScrapingMDS/issue...
26  https://github.com/pescap/WebScrapingMDS/issue...
29  https://github.com/pescap/WebScrapingMDS/issue...
32  https://github.com/pescap/WebScrapingMDS/issue...
35  https://github.com/pescap/WebScrapingMDS/issue...
38  https://github.com/pescap/WebScrapingMDS/issue...
40  https://github.com/pescap/WebScrapingMDS/issue...
42  https://github.com/pescap/WebScrapingMDS/issue...
44  https://github.com/pesca

In [7]:
df_Final = IssuesToDataFrame(pr)
df_Final

,issue,author,state,assignees
0,65,jwilenmann,open,[No one assigned]
0,64,gavalenz,open,[No one assigned]
0,62,jwilenmann,open,[No one assigned]
0,61,GeorgeAdrock,open,[No one assigned]
0,60,MiguelAbarzua,open,[No one assigned]
0,54,ronaldchester,open,[No one assigned]
0,50,Carloslugook,open,[No one assigned]
0,48,omaldonados,open,[No one assigned]
0,46,pescap,closed,[No one assigned]
0,43,DavidValenzuelaP,open,[No one assigned]
